In [1]:
import os
import math
import pandas as pd
import numpy as np
from data_functions import get_ant_datafiles_info, preprocess_antcolony_data


os.environ['QT_DEVICE_PIXEL_RATIO'] = ''  # disable annoying warnings on plotting

In [2]:
get_ant_datafiles_info()

,recording_location,file,location,n_ants,species_name,enterance
0,indoor,dataset/ants/IndoorDataset/Seq0001Object10Imag...,china,10,japanese_arched_ant,0
1,indoor,dataset/ants/IndoorDataset/Seq0002Object10Imag...,china,10,japanese_arched_ant,0
2,indoor,dataset/ants/IndoorDataset/Seq0003Object10Imag...,china,10,japanese_arched_ant,0
3,indoor,dataset/ants/IndoorDataset/Seq0004Object10Imag...,china,10,japanese_arched_ant,0
4,indoor,dataset/ants/IndoorDataset/Seq0005Object10Imag...,china,10,japanese_arched_ant,0
5,outdoor,dataset/ants/OutdoorDataset/Seq0006Object21Ima...,russia,73,carpenter_ant,1
6,outdoor,dataset/ants/OutdoorDataset/Seq0007Object26Ima...,greece,162,little_black_ant,1
7,outdoor,dataset/ants/OutdoorDataset/Seq0008Object31Ima...,russia,133,carpenter_ant,1
8,outdoor,dataset/ants/OutdoorDataset/Seq0009Object50Ima...,russia,193,carpenter_ant,1
9,outdoor,dataset/ants/OutdoorDataset/Seq0010Object30Ima...,usa,101,little_black_ant,0


In [3]:
data_info = get_ant_datafiles_info()
selected_files = data_info['file'].iloc[0:10]

for i, file_path in enumerate(selected_files):
    print(file_path)
    ants = preprocess_antcolony_data(file_path)

    # Generate a unique filename for each file
    # This assumes files have a readable structure and you can name them sequentially
    # name which are indoor and which are outdoor

    output_filename = f'dataset/ants/csvs/ants_seq{i+1}.csv'  
    ants.to_csv(output_filename)

    print(ants)

    
    print(f"Data written to {output_filename}")

dataset/ants/IndoorDataset/Seq0001Object10Image94/gt/gt.txt
      frameno  ant_id         x         y   next_x    next_y      speed  \
0           1      11   693.603    1.5630  674.258    8.8174  20.660478   
1           1      12   671.840  639.9510  674.258  644.7870   5.406812   
2           1      13   758.893  777.7850  746.802  777.7850  12.091000   
3           1      14   780.656  695.5680  778.238  690.7320   5.406812   
4           1      15   843.528  303.8300  845.946  296.5760   7.646387   
...       ...     ...       ...       ...      ...       ...        ...   
3505      351      16  1087.760  226.4500      NaN       NaN        NaN   
3506      351      17   855.618  -22.6184      NaN       NaN        NaN   
3507      351      18   962.020  199.8500      NaN       NaN        NaN   
3508      351      19  1128.870  378.7920      NaN       NaN        NaN   
3509      351      20   691.185   74.1070      NaN       NaN        NaN   

         angle  cos(angle)  
0    -0.35

In [4]:
import pandas as pd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.time_series import TimeSeries
from pyrqa.computation import RQAComputation

def load_and_preprocess_data(file_name):
    data = pd.read_csv(file_name)
    # Assuming 'x' and 'y' are the coordinates we're interested in
    return data['x'].values, data['y'].values  # Separate the dimensions for individual processing

def perform_RQA(time_series_data, radius=0.5, embedding_dimension=2, time_delay=1):
    time_series = TimeSeries(time_series_data, embedding_dimension=embedding_dimension, time_delay=time_delay)
    settings = Settings(time_series, neighbourhood=FixedRadius(radius))
    result = RQAComputation.create(settings).run()
    return result

# Iterate over your CSV files
for i in range(1, 11):
    file_name = f'dataset/ants/csvs/ants_seq{i}.csv'
    x_data, y_data = load_and_preprocess_data(file_name)

    # Analyze each dimension separately
    # print(f'Results for {file_name}:')
    # for dimension, data in zip(['x', 'y'], [x_data, y_data]):
    #     rqa_result = perform_RQA(data)
    #     print(f'Dimension {dimension}:')
    #     print(rqa_result)


In [5]:
# analyze on an ant level whether an ants position recurrs with iteself (potentially explaining behavior of a single ant)
import pandas as pd
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
import matplotlib as plt
from pyrqa.computation import RPComputation
from pyrqa.image_generator import ImageGenerator
import io
from pyrqa.analysis_type import Cross
import random
from multiSyncPy import synchrony_metrics as sm

In [7]:
file0 = 'dataset/ants/SurrogateTrajectoriesDataset/surrogate_data_colony_0.csv'
data0 = pd.read_csv(file0)

# Define a new function to correctly convert stringified lists with missing commas
def correct_and_convert(string):
    # Remove square brackets and split by space
    string = string.strip('[]')
    number_strings = string.split()
    # Convert each part to float
    return [float(num) for num in number_strings]

# Apply the conversion function to both columns
data0['surrogate_x'] = data0['surrogate_x'].apply(correct_and_convert)
data0['surrogate_y'] = data0['surrogate_y'].apply(correct_and_convert)

# Display the transformed data
data0.head()


,ant_id,surrogate_x,surrogate_y
0,11,"[0.40033805, 0.36247493, 0.34801186, 0.3383863...","[-1.4821072, -1.46096676, -1.36172315, -1.3146..."
1,12,"[-0.322611082, -0.313614133, -0.315435163, -0....","[-0.734027122, -0.608921703, -0.556133932, -0...."
2,13,"[0.889451345, 0.882438854, 0.865082132, 0.7938...","[1.42578414, 1.28347449, 1.12055163, 1.0951149..."
3,14,"[-0.34977951, -0.35902492, -0.36791712, -0.367...","[0.24577948, 0.20217781, 0.19276916, 0.1954821..."
4,15,"[-1.93058527, -1.86145025, -1.82890262, -1.461...","[0.377023551, 0.220087758, 0.370870184, 0.2278..."


In [13]:

results0 = {}

# Assuming 'data0' is your DataFrame loaded and cleaned as we did previously
ants = data0
unique_ants = ants['ant_id'].unique()
selected_ants = np.random.choice(unique_ants, size=len(data0['ant_id']), replace=False)

for ant_id in selected_ants:
    ant_data = ants[ants['ant_id'] == ant_id].dropna(subset=['surrogate_x', 'surrogate_y'])
    
    # Assuming the data in 'surrogate_x' and 'surrogate_y' are already clean and in the correct format
    surrogate_x_y = np.column_stack((ant_data['surrogate_x'].explode().values,
                                     ant_data['surrogate_y'].explode().values))
    
    # Create time series object for RQA
    time_series = TimeSeries(surrogate_x_y, embedding_dimension=2, time_delay=2)
    
    settings = Settings(time_series, neighbourhood=FixedRadius(0.2), similarity_measure=EuclideanMetric(), theiler_corrector=1)
    
    result = RQAComputation.create(settings).run()
    results0[(ant_id)] = result

for key, value in results0.items():
    print(f"Results for Ant ID {key}, RQA: {value}")



c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Results for Ant ID 17, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.047085
Determinism (DET): 0.414779
Average diagonal line length (L): 4.540650
Longest diagonal line length (L_max): 29
Divergence (DIV): 0.034483
Entropy diagonal lines (L_entr): 1.992028
Laminarity (LAM): 0.048474
Trapping time (TT): 2.926316
Longest vertical line length (V_max): 7
Entropy vertical lines (V_entr): 1.309557
Average white vertical line length (W): 56.596872
Longest white vertical line length (W_max): 342
Longest white vertical line length inverse (W_div): 0.002924
Entropy white vertical lines (W_entr): 3.748357

Ratio determinism / recurrence rate (DET/RR): 8.809155
Ratio laminarity / determinism (LAM/DET): 0.116868

Results for Ant ID 11, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2


In [12]:
file7 = 'dataset/ants/SurrogateTrajectoriesDataset/surrogate_data_colony_7.csv'
data7 = pd.read_csv(file7)

# Define a new function to correctly convert stringified lists with missing commas
def correct_and_convert(string):
    # Remove square brackets and split by space
    string = string.strip('[]')
    number_strings = string.split()
    # Convert each part to float
    return [float(num) for num in number_strings]

# Apply the conversion function to both columns
data7['surrogate_x'] = data7['surrogate_x'].apply(correct_and_convert)
data7['surrogate_y'] = data7['surrogate_y'].apply(correct_and_convert)


results7 = {}

# Assuming 'data0' is your DataFrame loaded and cleaned as we did previously
ants = data7
unique_ants = ants['ant_id'].unique()
selected_ants = np.random.choice(unique_ants, size=len(data7['ant_id']), replace=False)

for ant_id in selected_ants:
    ant_data = ants[ants['ant_id'] == ant_id].dropna(subset=['surrogate_x', 'surrogate_y'])
    
    # Assuming the data in 'surrogate_x' and 'surrogate_y' are already clean and in the correct format
    surrogate_x_y = np.column_stack((ant_data['surrogate_x'].explode().values,
                                     ant_data['surrogate_y'].explode().values))
    
    # Create time series object for RQA
    time_series = TimeSeries(surrogate_x_y, embedding_dimension=2, time_delay=2)
    
    settings = Settings(time_series, neighbourhood=FixedRadius(0.2), similarity_measure=EuclideanMetric(), theiler_corrector=1)
    
    result = RQAComputation.create(settings).run()
    results7[(ant_id)] = result

for key, value in results7.items():
    print(f"Results for Ant ID {key}, RQA: {value}")

c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Results for Ant ID 78, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.070892
Determinism (DET): 0.479891
Average diagonal line length (L): 8.100917
Longest diagonal line length (L_max): 116
Divergence (DIV): 0.008621
Entropy diagonal lines (L_entr): 2.432887
Laminarity (LAM): 0.042146
Trapping time (TT): 6.600000
Longest vertical line length (V_max): 9
Entropy vertical lines (V_entr): 1.596098
Average white vertical line length (W): 55.512673
Longest white vertical line length (W_max): 233
Longest white vertical line length inverse (W_div): 0.004292
Entropy white vertical lines (W_entr): 4.325131

Ratio determinism / recurrence rate (DET/RR): 6.769348
Ratio laminarity / determinism (LAM/DET): 0.087823

Results for Ant ID 62, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

In [14]:
file8 = 'dataset/ants/SurrogateTrajectoriesDataset/surrogate_data_colony_8.csv'
data8 = pd.read_csv(file8)

# Define a new function to correctly convert stringified lists with missing commas
def correct_and_convert(string):
    # Remove square brackets and split by space
    string = string.strip('[]')
    number_strings = string.split()
    # Convert each part to float
    return [float(num) for num in number_strings]

# Apply the conversion function to both columns
data8['surrogate_x'] = data8['surrogate_x'].apply(correct_and_convert)
data8['surrogate_y'] = data8['surrogate_y'].apply(correct_and_convert)


results8 = {}

# Assuming 'data0' is your DataFrame loaded and cleaned as we did previously
ants = data8
unique_ants = ants['ant_id'].unique()
selected_ants = np.random.choice(unique_ants, size=len(data8['ant_id']), replace=False)

for ant_id in selected_ants:
    ant_data = ants[ants['ant_id'] == ant_id].dropna(subset=['surrogate_x', 'surrogate_y'])
    
    # Assuming the data in 'surrogate_x' and 'surrogate_y' are already clean and in the correct format
    surrogate_x_y = np.column_stack((ant_data['surrogate_x'].explode().values,
                                     ant_data['surrogate_y'].explode().values))
    
    # Create time series object for RQA
    time_series = TimeSeries(surrogate_x_y, embedding_dimension=2, time_delay=2)
    
    settings = Settings(time_series, neighbourhood=FixedRadius(0.2), similarity_measure=EuclideanMetric(), theiler_corrector=1)
    
    result = RQAComputation.create(settings).run()
    results8[(ant_id)] = result

for key, value in results8.items():
    print(f"Results for Ant ID {key}, RQA: {value}")

c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Results for Ant ID 18, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.072271
Determinism (DET): 0.440890
Average diagonal line length (L): 9.238612
Longest diagonal line length (L_max): 120
Divergence (DIV): 0.008333
Entropy diagonal lines (L_entr): 2.824571
Laminarity (LAM): 0.105674
Trapping time (TT): 6.678344
Longest vertical line length (V_max): 36
Entropy vertical lines (V_entr): 2.228447
Average white vertical line length (W): 96.226210
Longest white vertical line length (W_max): 513
Longest white vertical line length inverse (W_div): 0.001949
Entropy white vertical lines (W_entr): 4.231219

Ratio determinism / recurrence rate (DET/RR): 6.100478
Ratio laminarity / determinism (LAM/DET): 0.239684

Results for Ant ID 101, RQA: RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min):